# Inhaltsverzeichnis
1. Bibliotheken importieren
1. Anomalien entfernen und Feature Engeneering
1. Merkmale auswählen
1. Regression
1. Modelle Bewerten
1. Erklärung Lineare Regression
1. Evaluation GBT
1. Unabhängige Validierung
1. Anleitung

*Um das Model zu testen, ändern Sie den Dateinamen im Code-Teil unter "8.Unabhängige Validierung".*

# 1. Bibliotheken importieren

In [ ]:
# Data Science + Mathe Bibliotheken
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb

# Metriken
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import max_error

# Validierung
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# Modelle
from sklearn import preprocessing 
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor

# Warnungen 
import warnings

#define Mape
# https://www.statology.org/mape-python/
def mean_absolute_percentage_error(actual, pred):
    actual, pred = np.reshape(np.array(actual), (-1)), np.reshape(np.array(pred),(-1))
    return np.mean(np.abs((actual - pred) / actual)) * 100

# 2. Anomalien entfernen und Feature Engeneering

In [ ]:
housing_data = pd.read_csv('DatenAusgegeben1.2_UTF8_manuell.csv', sep=";", keep_default_na=False)

warnings.filterwarnings('ignore')

# Bereinigung und Feature Engeneering

# für €/qm² für besseren Vergleich der Daten
housing_data["Preisproqm"] = housing_data["Preis"]/housing_data["Wohnflaeche in qm"]

# Monate seit 2136
housing_data["Monate"] = (housing_data["Verkaufsjahr"]-2136)*12 + housing_data["Verkaufsmonat"] -1

# Werte der Heizungsqualitaet in Zahlen umwandeln
housing_data["Heizungsqualitaet"] = [1 if x=="Schl" else 2 if x=="Ud" else 3 if x=="Ty" else 4 if x=="Gut" else 5 for x in housing_data["Heizungsqualitaet"]]

# Zustand, 1-10 wird auf 1-5 gemapt, das ist zwar ein bisschen ungenauer, aber einfacher zu vergleichen
housing_data["Zustandf"] = [int(x/2) + x%2 for x in housing_data["Zustand"]]

# Alle Zustände werden von 1...X auf 0....X-1 geschoben
housing_data["HeizungsqualitaetN"] = [ x-1 for x in housing_data["Heizungsqualitaet"]]
housing_data["ZustandN"] = [ x-1 for x in housing_data["Zustandf"]]
housing_data["FassadeN"] = [ x-1 for x in housing_data["Zustand Fassade"]]
housing_data["KuecheN"] = [ x-1 for x in housing_data["Kuechenqualitaet"]]

# Klimaanlage Ja/Nein  -> 1/0
housing_data["Klimaanlage"] = [1 if x == "Y" else 0 for x in housing_data["Klimaanlage"]]

# Besonders Große Daten beim Preis + Wohnfläche in qm rausschmeißen
max_value = housing_data["Preis"].unique()[-1:][0]
housing_data[housing_data["Preis"] != max_value]

max_value = housing_data["Wohnflaeche in qm"].unique()[-1:][0]
housing_data[housing_data["Wohnflaeche in qm"] != max_value]

max_value = housing_data["Grundstueck in qm"].unique()[-2:][0]
housing_data[housing_data["Grundstueck in qm"] != max_value]


# NA in Garage Typ wird zu "Keine Garage"
housing_data["Garage Typ"] = ['keine Garage' if x=='NA' else x for x in housing_data["Garage Typ"]]


print(housing_data.keys())

# 3. Merkmale auswählen

In [ ]:
# X und Y aus dem Datenset auslesen

# Zu testende Merkmale festlegen (X)
features = [ "Grundstueck in qm", "Wohnflaeche in qm","ZustandN","KuecheN", "HeizungsqualitaetN", "FassadeN",
            "Klimaanlage", "Gebaut", "Garagenkapazitaet"]
# Räume 
# Preis (Y)
predict= "Preis"

X = {}
for feature in features:
    X[feature] = housing_data[feature]
X = pd.DataFrame(X)

Y = pd.DataFrame(housing_data[predict])

# 4. Regression

In [ ]:
# Finden der Hyperparameter für GBT

def info(data):
    best = {}
    for i in score_types_test:
        # Beste Scores der Modelle  auswählen
        if i == "r2score":
            best[i] = np.sort(data[i])[-1]
        else:
            best[i] = np.sort(data[i])[0]
    return best
        

def score_model_test(curr_model, x_test, y_test,scores):
    scores["r2score"].append(r2_score(curr_model.predict(x_test), y_test))
    scores["mse"].append(mean_squared_error(curr_model.predict(x_test), y_test))
    scores["rmse"].append(mean_squared_error(curr_model.predict(x_test), y_test, squared = False))
    scores["mape"].append(mean_absolute_percentage_error(curr_model.predict(x_test), y_test))
    scores["max"].append(max_error(curr_model.predict(x_test), y_test))
    return

def work(depth, estimators, learn,data):
    
    scores = {"r2score":[], "mse":[], "rmse":[], "mape":[], "max":[]}
    #Initialisierung
    for k in data:
        # Parameter durch probieren herausfinden 
        curr_model = GradientBoostingRegressor(
            max_depth=depth, n_estimators=estimators, learning_rate=learn/100, loss="ls")
        
        curr_model.fit(k["x_poly_train"], k["y_train"])
        score_model_test(curr_model, k["x_poly_test"], k["y_test"], scores)
    
    return scores
    
def update(best, best2):
    if len(best) == 0:
        #Start
        for i in best2:
            best[i] = {"value": best2[i], "depth" : depth, "n_est": n_est, "learn":learn}
    else:
        for i in best:
            if i == "r2score" and best[i]["value"] < best2[i]:
            # R2, der muss höher sein
                best[i] = {"value": best2[i], "depth" : depth, "n_est": n_est, "learn":learn} 
            elif best[i]["value"] > best2[i]:
                # sonst müssen sie kleiner sein
                best[i] = {"value": best2[i], "depth" : depth, "n_est": n_est, "learn":learn} 
    return

def init():
    # Daten vorbereiten
    data = []
    Kfold = KFold(n_splits=10, random_state=89)
    kfold = Kfold.split(X, Y)
    for k, (train, test) in enumerate(kfold):
        poly = PolynomialFeatures(degree=3)
        x_train = X.iloc[train, :]
        x_test = X.iloc[test, :]
        x_poly = poly.fit_transform(x_train)
        x_poly_test = poly.fit_transform(x_test)
        y_train = Y.iloc[train]
        y_test = Y.iloc[test]
        data.append({"x_poly_train": x_poly, "x_train": x_train, "y_train": y_train,
                     "x_poly_test": x_poly_test, "x_test": x_test, "y_test": y_test})
    # Modelle testen
    score_types_test = ["r2score", "mse", "rmse", "mape", "max"]
    best = {}
    # Start 
    for depth in range(3,7):
        for n_est in range(20,180,20):
            for learn in range(1,50,5):
                mscores = work(depth,n_est,learn,data)
                best2 = info(mscores)
                update(best, best2)
    print(best)
    return

init()

In [ ]:
# K-fache Cross-Validierung, Polynomial Features


# Speichert (R2, MSE, RMSE, MAPE, MAX) in  model[score][mtype]..
def score_model(curr_model, x_test, y_test, model ,mtype):
    
    model["scores"][mtype]["r2score"].append(r2_score(curr_model.predict(x_test), y_test))
    model["scores"][mtype]["mse"].append(mean_squared_error(curr_model.predict(x_test), y_test))
    model["scores"][mtype]["rmse"].append(mean_squared_error(curr_model.predict(x_test), y_test, squared = False))
    model["scores"][mtype]["mape"].append(mean_absolute_percentage_error(curr_model.predict(x_test), y_test))
    model["scores"][mtype]["max"].append(max_error(curr_model.predict(x_test), y_test))
    return

split = 10
Kfold = KFold(n_splits=split, random_state=89)
kfold = Kfold.split(X, Y)

storage_poly = {"scores":{}, "models" : {}, "best" : {}}
storage_normal = {"scores":{}, "models" : {}, "best" : {}}
storage = {"data":[],"arch_types": ["poly","normal"], 
           "types":[ "osl", "lasso", "ridge", "gbt", "rf", "sgd"], 
           "score_types": ["r2score", "mse", "rmse", "mape", "max"], "poly": storage_poly, "normal":storage_normal}

#Initialisierung
for mtype in storage["types"]:
    storage_poly["scores"][mtype] = {}
    storage_normal["scores"][mtype] = {}
    for i in storage["score_types"]:
        storage_poly["scores"][mtype][i] = []
        storage_normal["scores"][mtype][i] = []
        
    storage_poly["models"][mtype] = []
    storage_normal["models"][mtype] = []
    

for k, (train, test) in enumerate(kfold):
    # Daten vorbereiten und für später speichern
    poly = PolynomialFeatures(degree=3)
    x_train = X.iloc[train, :]
    x_test = X.iloc[test, :]
    x_poly = poly.fit_transform(x_train)
    x_poly_test = poly.fit_transform(x_test)
    y_train = Y.iloc[train]
    y_test = Y.iloc[test]
    storage["data"].append({"xpoly": x_poly, "x_train": x_train, "y_train": y_train, 
                          "x_poly_test": x_poly_test, "x_test": x_test, "y_test": y_test})
    
    # Modelle Testen, dabei sowohl 
    for arch in storage["arch_types"]:
        model = storage[arch]
        
        # Parameter durch probieren herausgefunden 
        model["models"]["osl"].append(linear_model.LinearRegression(n_jobs=-1))
        model["models"]["lasso"].append(linear_model.Lasso(alpha=1.5, random_state=404))
        model["models"]["ridge"].append(linear_model.Ridge(alpha=1.5))
        model["models"]["gbt"].append(GradientBoostingRegressor(
            max_depth=4, n_estimators=100, learning_rate=0.15, loss="ls"))
        model["models"]["rf"].append(RandomForestRegressor(
            max_depth=10, n_estimators=10, criterion="mse",n_jobs=-1,max_features=None))
        model["models"]["sgd"].append(SGDRegressor())
        

        #model["models"]["poly"].append(poly)
        for mtype in storage["types"]:
            curr_model = model["models"][mtype][k]
            
            if arch == "poly":
                curr_model.fit(x_poly, y_train)
                score_model(curr_model, x_poly_test, y_test, model, mtype)
                
            else:
                curr_model.fit(x_train, y_train)
                score_model(curr_model, x_test, y_test, model, mtype)

print("Done")

# 5. Modelle Bewerten

In [ ]:
# Berechnet das beste Model und
# R2, MSE, RMSE, MAPE, MAX
if "sgd" in storage["types"]:
            storage["types"].remove("sgd")

for arch in storage["arch_types"]:
    model = storage[arch]
    # Für jeden Modelltyp (pro arch_type) den besten Finden und alle Performancedaten auslesen
    for mtype in storage["types"]:
        # Bestes Modell nach R2 Wert auswählen
        best = model["scores"][mtype]["r2score"].index(np.sort(model["scores"][mtype]["r2score"])[-1:][0])
        model["best"][mtype] = best
        
        print(arch, mtype)
        for i in storage["score_types"]:
            mean_score = np.mean(model["scores"][mtype][i])
            std_score = np.std(model["scores"][mtype][i])
            # bis auf r2 ist kleiner besser
            if i == "r2score":
                max_score = np.sort(model["scores"][mtype][i])[-1]
            else:
                    max_score = np.sort(model["scores"][mtype][i])[0]
            
            print("%8s von %s:\t%.4f +- %.3f (%.4f best)" % (i,mtype, mean_score, std_score, max_score))

In [ ]:
#Todo -> WErte beschreiben, kurz!

In [ ]:
# Beispielgrafiken -> Entwicklung der Werte

def print_graphic(position, arch):
    model = storage[arch]
    # Eine Grafikreihe zeichnen
    mtype = storage["types"][i]
    best_model = model["models"][mtype][model["best"][mtype]]
    
    x_test = storage["data"][model["best"][mtype]]["x_test"]
    if arch == "poly":
        # Andere Eingaben, wenn Polynomielle Modelle verglichen werden.
        x_test = storage["data"][model["best"][mtype]]["x_poly_test"]
        
    y_test = storage["data"][model["best"][mtype]]["y_test"]
    pred_y = best_model.predict(x_test)
    # Abweichung in Prozent in Grafik packen
    # (1. Grafik)
    
    ax = fig.add_subplot(2,len(storage["types"]), position+1)
    plt.sca(ax)
    plt.hist(np.reshape(pred_y,(-1,1)) / np.reshape(y_test,(-1,1)))
    plt.xlim(0.25,1.75)
    plt.ylim(0,120)
    plt.title(storage["types"][position])
    plt.xlabel("Verhältnis der Werte")
    plt.ylabel("Anzahl")
    if position != 0:
        plt.yticks([])
    
    
    # Beispielvorhersage für ein Haus
    # (2. Grafik)
    x = np.arange(100,200,1)
    y = []
    for c in range(5): # 5 Zustände
        y.append([])
        for v in x: # für jeden x-Wert ein y-Wert berechnen
            
            #[ "Grundstueck in qm", "Wohnflaeche in qm","ZustandN","KuecheN", "HeizungsqualitaetN", "FassadeN",
            #"Klimaanlage", "Gebaut", "Garagenkapazitaet"]
            test_data = [[800 + 3*v, v, c, c, c, c, int(c/3), 2030, 2]]
            
            if arch == "poly":
                y[c].append(best_model.predict(poly.fit_transform(test_data))[0])
            else:
                y[c].append(best_model.predict(test_data)[0])
    
    #Plot
    ax = fig.add_subplot(2,len(storage["types"]), position+1+len(storage["types"]))
    plt.sca(ax)
    plt.xlim(100,200)
    plt.plot(x, y[0], c="yellow", label="1")     
    plt.plot(x, y[1], c="orange", label="2")
    plt.plot(x, y[2], c="red", label="3")
    plt.plot(x, y[3], c="purple" ,label="4")
    plt.plot(x, y[4], c="blue" ,label="5")
    plt.legend(loc="upper left", ncol=2)
    plt.title(storage["types"][position])
    plt.xlabel("Wohnflaeche in Qm")
    plt.ylabel("Preis")
    if position != 0:
        plt.yticks([])
    return

# Nachdem sgd zu schlecht ist, wird es zur Evaluation entfernt!


# 
for arch in storage["arch_types"]:
    
    # Grafiken vorbereiten
    fig, big_axes = plt.subplots( figsize=(15, 10) , nrows=2, ncols=1)

    big_axes[0].set_title("Verhältnis der vorhergesagten und wirklichen Preise",fontsize=16, pad=25)
    big_axes[0].tick_params(labelcolor=(1.,1.,1., 0.0), top='off', bottom='off', left='off', right='off')
    big_axes[0]._frameon = False

    big_axes[1].set_title("Preisentwickung bei verschiedenen Hauszuständen",fontsize=16,pad=25)
    big_axes[1].tick_params(labelcolor=(1.,1.,1., 0.0), top='off', bottom='off', left='off', right='off')
    big_axes[1]._frameon = False
    
    #Grafiken zeichnen
    for i in range(len(storage["types"])):
        print_graphic(i,arch)
        
        

    plt.tight_layout()
    plt.show()

Wie in den Graphen zu sehen ist, entspricht das Verhältnis der vorhergesagten und wahren Preisen einer Gaussverteilung, was zu erwarten war. Nachdem der Stochastic Gradiend Descend zu schlechte Werte vorhergesagt hat, wurde in der Analyse nicht weiter betrachtet. Ein Grund dafür kann sein, dass der SGD für besonders große Datenmengen gedacht ist. Desweiteren kommt die Ordinary Squared Loss und die ridge-Regression nicht mit einer Input-Transformation auf ein Polynom #TODO 4. Grades zurecht, deren Werte sind bei linearen Eingaben deutlich besser.    
In der zweiten Graphenreihe wurde ein fiktives Haus erstellt, dessen Zustand und Wohnfläche variabel ist. In den Graphiken ist gut zusehen, dass es einen großen preislichen Unterschied zwischen den einzelnen Zuständen gibt. Sehr gute Häuser sind deutlich teurer. In der Ordinarily Squared Loss- und Ridge-Regression werden Häuser mit sehr niedrigen Qualität bei größer werdender Wohnfläche billiger, was entweder auf ein schlechtes Modell oder auf schlechte Testdaten hindeutet. Die Relation von Grundstücksfläche = 800 + 3 * Wohnfläche kann auch Einfluss auf die Qualität der Vorhersage nehmen. 

# Erklärung Lineare Regression

In [ ]:
Ein leicht Verständliches Modell ist die Ordinary Squared Loss-Regression mit linearen Eingaben. 
Unten wird die Formel ausgegeben, mit der Vorsagen getroffen werden.

In [ ]:
linreg = storage["normal"]["osl"][storage["normal"]["best"]["osl"]]
coefs = linreg.coef_
features
formula = "y(X) = "
for i in range(len(features)):
    fomula += " %d * \"%s\" +" % (coefs[i], features[i])
formula = formula[:-1]
print(formula)

# 6. Evaluation GBT

In diesem Projekt wurden folgende Regressionen getestet: 
* Ordinary Squared Loss 
* Lasso 
* Ridge 
* Gradient Boosting Trees 
* Random Forest<br>
<br>
Die Modelle wurden mit 1600 Wohnungsdatensätzen trainiert. Diese Merkmale wurden als Eingabe verwendet:
* Grundstück in qm 
* Wohnfläche in qm
* Zustand         (Nullbasiert)
* Zustand Kueche  (Nullbasiert)
* Zustand Heizung (Nullbasiert)
* Zustand Fassade (Nullbasiert)
* Klimaanlage
* Baujahr
* Garagenkapazität
* Anzahl Räume
<br>
<br>
Mit diesen Merkmalen wird von den verwendeten Modellen der Preis in GC Dollar vorhergesagt.<br>
<br>
Bei diesem Projekt hat sich Gradient Boosting Trees als bestes Model herausgestellt.<br>
Mit einem maximalen R2-Wert von 0,884 hat diesen Model nach Random Forest (0,894) und Lasso (0,889) den dritthöchsten Wert in den zehn Trainingsdurchläufen (K-fache Kreuz-Validierung) erreicht.<br>
Jedoch hatte Gradient Boosting Trees die geringste Standardabweichung im R2-Wert (+- 0,031) in allen Durchgängen und damit auch den besten durchschnittlichen R2-Wert (0,837).<br>
Mit einem Root Mean Square Error von 20.190,43, liegt das Modell mit seinen Prognosen im Mittel ca. 20.000 GC Dollar daneben. <br>
Bei einem Maximum Error von 65422,66 liegt die größte Abweichung im Test bei ca. 65.000 GC Dollar. <br>
Bei beiden dieser Metriken ist Gradient Boosting Trees der klare Sieger. <br>
Es muss jedoch beachtet werden, dass das Model mit 38,59 einen sehr hohen Mean Absolute Percentage Error hat.
#TODO Gradiend boosting tree ausreißer
#TOdo Grafik oben bewerten

Summary (3 - 5 Punkte)

# 7. Unabhängige Validierung

In [ ]:
housing_data2 = pd.read_csv('DatenAusgegeben1.2_UTF8_manuell.csv', sep=";", keep_default_na=False)


housing_data2["Preisproqm"] = housing_data2["Preis"]/housing_data2["Wohnflaeche in qm"]

# Monate seit 2136
housing_data2["Monate"] = (housing_data2["Verkaufsjahr"]-2136)*12 + housing_data2["Verkaufsmonat"] -1

# Werte der Heizungsqualitaet in Zahlen umwandeln
housing_data2["Heizungsqualitaet"] = [1 if x=="Schl" else 2 if x=="Ud" else 3 if x=="Ty" else 4 if x=="Gut" else 5 for x in housing_data2["Heizungsqualitaet"]]

# Zustand, 1-10 wird auf 1-5 gemapt, das ist zwar ein bisschen ungenauer, aber einfacher zu vergleichen
housing_data2["Zustandf"] = [int(x/2) + x%2 for x in housing_data2["Zustand"]]

# Alle Zustände werden von 1...X auf 0....X-1 geschoben
housing_data2["HeizungsqualitaetN"] = [ x-1 for x in housing_data2["Heizungsqualitaet"]]
housing_data2["ZustandN"] = [ x-1 for x in housing_data2["Zustandf"]]
housing_data2["FassadeN"] = [ x-1 for x in housing_data2["Zustand Fassade"]]
housing_data2["KuecheN"] = [ x-1 for x in housing_data2["Kuechenqualitaet"]]

# Klimaanlage Ja/Nein  -> 1/0
housing_data2["Klimaanlage"] = [1 if x == "Y" else 0 for x in housing_data2["Klimaanlage"]]

# NA in Garage Typ wird zu "Keine Garage"
housing_data2["Garage Typ"] = ['keine Garage' if x=='NA' else x for x in housing_data2["Garage Typ"]]


X2 = {}
for feature in features:
    X2[feature] = housing_data2[feature]
X2 = pd.DataFrame(X)

Y2 = pd.DataFrame(housing_data2[predict])

x2_poly = poly.fit_transform(X2.iloc[train, :])
poly.fit(X2.iloc[train, :], Y2.iloc[train])
x2_poly_test = poly.fit_transform(X2.iloc[test, :])

# R2, MSE, RMSE, MAPE, MAX
test_model = model["models"]["gbt"][model["best"]["gbt"]]
x_test = model["data"][model["best"]["gbt"]]["x_poly_test"]
y_test = model["data"][model["best"]["gbt"]]["y_test"]

r2_test = round(r2_score(test_model.predict(x_test), y_test), 2)
mse_test = round(mean_squared_error(test_model.predict(x_test), y_test), 2)
rmse_test = round(mean_squared_error(test_model.predict(x_test), y_test, squared=False), 2)
mape_test = round(mean_absolute_percentage_error(test_model.predict(x_test), y_test), 2)
max_test = round(max_error(test_model.predict(x_test), y_test), 2)

print("R2 Wert:", r2_test)
print("Mean Squared Error Wert:", mse_test)
print("Root Mean Square Error Wert:", rmse_test)
print("Mean Absolute Percentage Error Wert:", mape_test)
print("Maximum Error Wert:", max_test)

# 8. Anleitung

In [ ]:
def mpredict(data, renovation=None):
    mtyp = "gbt"
    
    dinput = ["Grundstück", "Wohnraum", "Zustand", "Küche", "Klimaanlage", "Fassade", "Gebaut", "Garagenkapazitaet", "Heizungsqualitaet", "Räume"]
    
    curr_model = model["models"][mtyp][model["best"][mtyp]]
    # Eingaben überprüfen
    if len(data) != len(dinput):
        print("Eingaben überprüfen: %d Elemente erwartet, aber nur %d Elemente vorhanden!" % 
              (len(dinput), len(data)))
        return
    
    
    pred_y = curr_model.predict(poly.fit_transform([data]))[0]
    print("Vorhergesagter Preis: %i€" % int(pred_y))
    
    
    if renovation != None:
        new_data = data.copy()
        # neue Daten laden
        for ren_data in renovation.keys():
            if ren_data not in dinput:
                print("Unerkannte Eingabe, Renovation überprüfen")
                return;
            index = dinput.index(ren_data)
            new_data[index] = renovation[ren_data]
        
        # Evaluieren
        pred_y_new = curr_model.predict(poly.fit_transform([new_data]))[0]
        print("Vorhergesagter Wert nach Renovation: %d€, Differenz: %d€" % (pred_y_new, pred_y_new - pred_y))

Anhand der vorhandenen Daten wurde ein Modell entwickelt, mit dem der preisliche Aufwand von Renovationen mit der Wertsteigerung der Immobilie verglichen werden kann. Für diese Aufgabe wurden Modelle auf 1600 Datensätzen trainiert, getestet und die Besten für eine weitere Untersuchung ausgewählt. Das beste Modell war ein Gradiend Boosting Tree (GBT) mit einem R2-Wert von 0.88 #TODO, ist das noch richtig?
Das Modell bietet herausragende Vorhersagegenauigkeit für Häuser im Preisbereich von 100.000 bis 500.000 GC Dollar. Sollten Werte getestet werden, die außerhalb dieses Bereichs liegen, 

werden zwar immernoch gute Preisvorschläge generiert, ihre Richtigkeit kann aber nicht mehr garantiert werden.

Um eine Vorhersage zu erhalten, werden folgende Daten benötigt:
- Grundstücksfläche in Qm
- Wohnfläche in Qm
- Zustand des Hauses (Zahl von 0 - 4)
- Zustand der Küche (Zahl von 0 - 4)
- Vorhandensein einer Klimaanlage (0: Nein, 1: Ja)
- Baujahr
- Garagenkapazität
- Zustand der Heizung (Zahl von 0 - 4)
- Zustand der Fassade (Zahl von 0 - 4)
- Anzahl der Räume

ein Beispielaufruf kann so aussehen:    
daten = [1200,80,4,0,0,4,1,2103,4,4]    
renovation =  {"Küche": 4, "Klimaanlage": 1}  
mpredict(daten, renovation) 


Mit diesem Tool kann nun der Wer der Immobilie nach einer Renovation berechnet werden, indem dem Modell der gewünschte Zielzustand gegeben wird. Das Modell berechnet nun den neuen Verkaufspreis und gibt dazu die Differenz zum jetzigen Preis aus. Mit diesem Preis kann nun die Wirtschaftlichkeit der Renovation geprüft und ein Kaufentscheid gefällt werden. # TODO darüber schauen

In [ ]:
#"Grundstueck in qm", "Wohnflaeche in qm","ZustandN","KuecheN","Klimaanlage", "Gebaut", #"Garagenkapazitaet", "HeizungsqualitaetN", "Raeume"]
# Istpreis s Preis nach Renovation
daten = [1800,140,4,1,1,4,1,2203,2,4]
renovation =  {"Küche": 4}
mpredict(daten, renovation)